# LSTM Stock Predictor Using Fear and Greed Index

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [153]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [154]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [155]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [156]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [157]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [158]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [159]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [160]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [161]:
# Use 70% of the data for training and the remaineder for testing
# YOUR CODE HERE!
percent = int(len(X) * 0.7)

X_train = X[: percent - 1]
y_train = y[: percent - 1]

X_test = X[percent:]
y_test = y[percent:]

In [162]:
from sklearn.preprocessing import MinMaxScaler
# Use the MinMaxScaler to scale data between 0 and 1.
# YOUR CODE HERE!

y_test_scaler = MinMaxScaler()

y_test_scaler.fit(X)
X_train = y_test_scaler.transform(X_train)
X_test = y_test_scaler.transform(X_test)

y_test_scaler.fit(y)
y_train = y_test_scaler.transform(y_train)
y_test = y_test_scaler.transform(y_test)

X_train[:5]


array([[0.60761794, 0.58242373, 0.62172321, 0.51499412, 0.38267307,
        0.46160996, 0.4503991 , 0.5193988 , 0.56445096, 0.55127638],
       [0.58242373, 0.62172321, 0.51499412, 0.38267307, 0.46160996,
        0.4503991 , 0.5193988 , 0.56445096, 0.55127638, 0.5012101 ],
       [0.62172321, 0.51499412, 0.38267307, 0.46160996, 0.4503991 ,
        0.5193988 , 0.56445096, 0.55127638, 0.5012101 , 0.58660203],
       [0.51499412, 0.38267307, 0.46160996, 0.4503991 , 0.5193988 ,
        0.56445096, 0.55127638, 0.5012101 , 0.58660203, 0.5487353 ],
       [0.38267307, 0.46160996, 0.4503991 , 0.5193988 , 0.56445096,
        0.55127638, 0.5012101 , 0.58660203, 0.5487353 , 0.64593307]])

In [163]:
# Reshape the features for the model
# YOUR CODE HERE!
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")

X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.60761794]
  [0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]]

 [[0.58242373]
  [0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]]

 [[0.62172321]
  [0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]]

 [[0.51499412]
  [0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]]

 [[0.38267307]
  [0.46160996]
  [0.4503991 ]
  [0.5193988 ]
  [0.56445096]
  [0.55127638]
  [0.5012101 ]
  [0.58660203]
  [0.5487353 ]
  [0.64593307]]] 

X_test sample values:
[[[0.04675556]
  [0.03913222]
  [0.03975922]
  [0.03856719]
  [0.03679562]
  [0.03728835]
  [0.03974167]
  [0.04528668]
  [0.04528668]
  [0.07024855]]

 [[0.03913222]
  [0.03975922]
  [0.038567

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [164]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [165]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# YOUR CODE HERE!

LSTM_model = Sequential()
number_units = 50
dropout_fraction = 0.3

# First Layer
LSTM_model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
LSTM_model.add(Dropout(dropout_fraction))

# Second Layer
LSTM_model.add(LSTM(units=number_units, return_sequences=True))
LSTM_model.add(Dropout(dropout_fraction))

# Third Layer
LSTM_model.add(LSTM(units=number_units))
LSTM_model.add(Dropout(dropout_fraction))

# Final Layer
LSTM_model.add(Dense(1))

In [166]:
# Compile the model
# YOUR CODE HERE!
LSTM_model.compile(optimizer="adam", loss="mean_squared_error")

In [167]:
# Summarize the model
# YOUR CODE HERE!
LSTM_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 10, 50)            10400     
_________________________________________________________________
dropout_21 (Dropout)         (None, 10, 50)            0         
_________________________________________________________________
lstm_22 (LSTM)               (None, 10, 50)            20200     
_________________________________________________________________
dropout_22 (Dropout)         (None, 10, 50)            0         
_________________________________________________________________
lstm_23 (LSTM)               (None, 50)                20200     
_________________________________________________________________
dropout_23 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                

In [168]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!
LSTM_model.fit(
    X_train, 
    y_train, 
    shuffle=False, 
    batch_size=3, 
    epochs=30, 
    verbose=1
)

Epoch 1/30
124/124 [==============================] - 4s 7ms/step - loss: 0.0909
Epoch 2/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0622
Epoch 3/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0557
Epoch 4/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0413
Epoch 5/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0321
Epoch 6/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0326
Epoch 7/30
124/124 [==============================] - 1s 8ms/step - loss: 0.0311
Epoch 8/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0271
Epoch 9/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0294
Epoch 10/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0255
Epoch 11/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0221
Epoch 12/30
124/124 [==============================] - 1s 7ms/step - loss: 0.0219
Epoch 13/30
124/124 [====

---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [169]:
# Evaluate the model
# YOUR CODE HERE!
LSTM_model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 3ms/step - loss: 0.0073


0.007302311714738607

In [170]:
# Make some predictions
# YOUR CODE HERE!
predictions = LSTM_model(X_test)

In [171]:
# Recover the original prices instead of the scaled version
predicted_prices = y_test_scaler.inverse_transform(predictions)
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [172]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
2019-02-20,3924.239990,3582.527494
2019-02-21,3974.050049,3685.875902
2019-02-22,3937.040039,3781.471989
2019-02-23,3983.530029,3836.986109
2019-02-24,4149.089844,3862.537226


In [173]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!
%matplotlib inline

stocks.hvplot(
    title="Predictions vs. Real Stock Prices",
    ylabel="Price",
    xlabel="Date"
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)